In [7]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
from textual_analysis.tools_scrapping import gather_links

In [8]:
def clean_links(links):
    clean_links_array = []
    for link in links:
        if "quote" in link and "-USD" in link:
            clean_links_array.append(link.replace("/quote/",""))
            
    return clean_links_array

In [9]:
symbols = []

for offset in np.arange(0,9900,100):
    url = f"https://finance.yahoo.com/screener/predefined/all_cryptocurrencies_us/?count=100&offset={offset}"

    resp = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
        },
    )
    
    links = gather_links(resp)
    
    clean_links_array = clean_links(links)
    
    symbols = symbols + clean_links_array

/home/ignacio/Documents/SFU/crypto/textual_analysis/tools_scrapping.py:19: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 19 of the file /home/ignacio/Documents/SFU/crypto/textual_analysis/tools_scrapping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  for link in BeautifulSoup(response.content, parse_only=SoupStrainer('a')):


In [10]:
symbols_bis = symbols.copy()
symbols = []
for x in symbols_bis:
    if x not in symbols:
        symbols.append(x)

In [11]:
stablecoins = list(pd.read_csv('stablecoins.csv').keys())
stablecoins = list(map(lambda x: x.lower(), stablecoins))

In [12]:
symbol_non_stablecoins = []
symbol_stablecoins     = []

for symbol in symbols:
    
    cmp = symbol.replace("-USD", "").lower()
    if cmp in stablecoins:
        symbol_stablecoins.append(symbol)
    else:
        symbol_non_stablecoins.append(symbol)

In [52]:
print(len(symbol_non_stablecoins))
print(len(symbol_stablecoins))

7744
72


In [13]:
i = 0
for symbol in symbols:
    msft = yf.Ticker(symbol)
    temp_data = msft.history(period="max")
    
    if temp_data.empty:
        i+=1
        continue
    
    nrows = temp_data.shape[0]
    ticker = [symbol] * nrows
    temp_data = temp_data.drop(columns = ['Dividends', "Stock Splits"])
    temp_data['close_times_volume'] = temp_data['Close'] * temp_data['Volume']
    temp_data['Ticker'] = ticker
    temp_data['Date'] = temp_data.index
    
    if symbol in symbol_stablecoins:
        temp_data['stablecoin'] = [True]  * nrows
    else:
        temp_data['stablecoin'] = [False] * nrows
        
    if i == 0:
        data = temp_data.copy()
    else:
        data = pd.concat([data, temp_data],axis = 0)
    
    i+=1
    
compression_opts = dict(method='zip',
                        archive_name='data_crypto_25_april.csv')
data.to_csv('data_crypto_25_april.zip', index=False,
          compression=compression_opts) 

IKI-USD: Period 'max' is invalid, must be one of ['1d', '5d']
DILL-USD: Period 'max' is invalid, must be one of ['1d', '5d']
GUMMIES-USD: Period 'max' is invalid, must be one of ['1d', '5d']
GENAI31054-USD: Period 'max' is invalid, must be one of ['1d', '5d']
ROBO18457-USD: Period 'max' is invalid, must be one of ['1d', '5d']
CPA-USD: Period 'max' is invalid, must be one of ['1d', '5d']
KILLSOLANA-USD: Period 'max' is invalid, must be one of ['1d', '5d']
PIRATECOIN-USD: Period 'max' is invalid, must be one of ['1d', '5d']
IRIS26908-USD: Period 'max' is invalid, must be one of ['1d', '5d']
XBTC-USD: Period 'max' is invalid, must be one of ['1d', '5d']
BOCAT-USD: Period 'max' is invalid, must be one of ['1d', '5d']


In [15]:
data

,Open,High,Low,Close,Volume,close_times_volume,Ticker,Date,stablecoin
Date,,,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,9.629991e+09,BTC-USD,2014-09-17 00:00:00+00:00,False
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,1.463605e+10,BTC-USD,2014-09-18 00:00:00+00:00,False
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,1.497055e+10,BTC-USD,2014-09-19 00:00:00+00:00,False
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,1.507367e+10,BTC-USD,2014-09-20 00:00:00+00:00,False
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,1.060070e+10,BTC-USD,2014-09-21 00:00:00+00:00,False
...,...,...,...,...,...,...,...,...,...
2024-04-29 00:00:00+00:00,0.071031,0.073879,0.060258,0.060437,106445240,6.433231e+06,KNFT-USD,2024-04-29 00:00:00+00:00,False
2024-04-30 00:00:00+00:00,0.060397,0.061748,0.045054,0.048623,59956819,2.915280e+06,KNFT-USD,2024-04-30 00:00:00+00:00,False
2024-05-01 00:00:00+00:00,0.048583,0.053489,0.031196,0.031254,62400528,1.950266e+06,KNFT-USD,2024-05-01 00:00:00+00:00,False


In [55]:
len(data[data.stablecoin == False]['Ticker'].unique())

7729

In [56]:
len(data[data.stablecoin == True]['Ticker'].unique())

72

In [26]:
len(symbol_non_stablecoins)

9811

In [27]:
len(set(symbol_non_stablecoins))

7744